In [ ]:
import numpy as np
import pandas as pd
import time

from pandas.api.types import is_numeric_dtype

from scipy.stats import wilcoxon, norm

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, ParameterSampler, HalvingRandomSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import (
    f1_score, balanced_accuracy_score, matthews_corrcoef, cohen_kappa_score,
    make_scorer, precision_score, recall_score, accuracy_score, confusion_matrix
)

rng = np.random.RandomState(42)


In [2]:
df = pd.read_csv("forestCover.csv", na_values=["?"])
df = df.rename(columns={"Water_Level": "Observation_ID", "Observation_ID": "Water_Level"})

TARGET_COL = "Cover_Type"

y = df[TARGET_COL].copy()
X = df.drop(columns=[TARGET_COL]).copy()

print(f"Shapes | X: {X.shape} | y: {y.shape}")
print("\ndtypes head:\n", X.dtypes.head(12))

print("\nClass distribution:")
print(y.value_counts().sort_index().to_frame("count").assign(pct=lambda t: 100*t["count"]/len(y)).round(2))


Shapes | X: (581012, 58) | y: (581012,)

dtypes head:
 Elevation                               int64
Aspect                                  int64
Facet                                 float64
Slope                                 float64
Inclination                           float64
Horizontal_Distance_To_Hydrology        int64
Vertical_Distance_To_Hydrology          int64
Horizontal_Distance_To_Roadways         int64
Hillshade_9am                           int64
Hillshade_Noon                          int64
Hillshade_3pm                           int64
Horizontal_Distance_To_Fire_Points      int64
dtype: object

Class distribution:
             count    pct
Cover_Type               
1           211840  36.46
2           283301  48.76
3            35754   6.15
4             2747   0.47
5             9493   1.63
6            17367   2.99
7            20510   3.53


In [ ]:
from typing import List, Tuple, Dict

TRUE_TOKENS  = {"1","true","t","yes","y","pos","positive","on"}
FALSE_TOKENS = {"0","false","f","no","n","neg","negative","off"}

def is_semantic_binary(values: pd.Series) -> bool:
    #Returns True if it looks like a binary field
    uniq = pd.unique(values.dropna().astype(str).str.strip().str.lower())
    if len(uniq) != 2:
        return False
    a, b = uniq
    
    return ((a in TRUE_TOKENS and b in FALSE_TOKENS) or (a in FALSE_TOKENS and b in TRUE_TOKENS))

def infer_feature_types(frame: pd.DataFrame) -> Tuple[List[str], List[str], List[str]]:
    """
    Return (numeric_cols, binary_indicator_cols, non_binary_categoricals)
    """
    num_cols, bin_cols, cat_cols = [], [], []
    for c in frame.columns:
        s = frame[c]
        # Fast path for numeric dtypes
        if pd.api.types.is_numeric_dtype(s):
            uniq = pd.unique(s.dropna())
            if len(uniq) == 2 and set(pd.to_numeric(uniq, errors="coerce")) <= {0,1}:
                bin_cols.append(c)
            else:
                num_cols.append(c)
            continue

        # Object/string-like: check binary semantics first
        if is_semantic_binary(s):
            bin_cols.append(c)
        else:
            # If exactly two string levels but not recognized, still treat as binary
            uniq = pd.unique(s.dropna())
            if len(uniq) == 2:
                bin_cols.append(c)
            else:
                cat_cols.append(c)
    return num_cols, bin_cols, cat_cols

NUM_COLS, BIN_COLS, CAT_COLS = infer_feature_types(X)
print(f"Numeric: {len(NUM_COLS)}  |  Binary: {len(BIN_COLS)}  |  Non-binary categoricals: {len(CAT_COLS)}")
print("Numeric:")
for i in range(0, len(NUM_COLS), 4):
    print("  " + ", ".join(NUM_COLS[i:i+4]))
print("Binary:")
for i in range(0, len(BIN_COLS), 4):
    print("  " + ", ".join(BIN_COLS[i:i+4]))
print("Categorical: ", CAT_COLS)
if CAT_COLS:
    print("Non-binary categoricals (unexpected here, just flagging):", CAT_COLS[:10])


Numeric: 14  |  Binary: 44  |  Non-binary categoricals: 0
Numeric:
  Elevation, Aspect, Facet, Slope
  Inclination, Horizontal_Distance_To_Hydrology, Vertical_Distance_To_Hydrology, Horizontal_Distance_To_Roadways
  Hillshade_9am, Hillshade_Noon, Hillshade_3pm, Horizontal_Distance_To_Fire_Points
  Observation_ID, Water_Level
Binary:
  Wilderness_Area1, Wilderness_Area2, Wilderness_Area3, Wilderness_Area4
  Soil_Type1, Soil_Type2, Soil_Type3, Soil_Type4
  Soil_Type5, Soil_Type6, Soil_Type7, Soil_Type8
  Soil_Type9, Soil_Type10, Soil_Type11, Soil_Type12
  Soil_Type13, Soil_Type14, Soil_Type15, Soil_Type16
  Soil_Type17, Soil_Type18, Soil_Type19, Soil_Type20
  Soil_Type21, Soil_Type22, Soil_Type23, Soil_Type24
  Soil_Type25, Soil_Type26, Soil_Type27, Soil_Type28
  Soil_Type29, Soil_Type30, Soil_Type31, Soil_Type32
  Soil_Type33, Soil_Type34, Soil_Type35, Soil_Type36
  Soil_Type37, Soil_Type38, Soil_Type39, Soil_Type40
Categorical:  []


In [ ]:
#Check the data to confirm the data quality issues mentioned in the assignment
def audit_dataset(X: pd.DataFrame, y: pd.Series, num_cols, bin_cols, corr_threshold=0.95):
    report = {}

    # Missingness
    miss_per_col = X.isna().sum()
    miss_rows = X.isna().any(axis=1).sum()
    report["missing_cols_nonzero"] = miss_per_col[miss_per_col > 0].sort_values(ascending=False)
    report["missing_rows_nonzero"] = miss_rows

    # Duplicates
    dup_rows = X.duplicated().sum()
    report["duplicate_rows"] = dup_rows

    # Zero and near-zero variance (binary and numeric separately)
    def nzv_mask(series, tol=1e-12):
        # Treat as zero-variance if all values equal; only compute std for numeric dtypes
        if series.nunique(dropna=False) == 1:
            return True
        if pd.api.types.is_numeric_dtype(series):
            return series.std(skipna=True) < tol
        return False

    zero_var_cols = [c for c in X.columns if nzv_mask(X[c])]
    report["zero_variance_cols"] = zero_var_cols

    # Near-zero variance heuristic for numeric only (IQR ~ 0)
    nzv_numeric = []
    for c in num_cols:
        q1, q3 = X[c].quantile([0.25, 0.75])
        if pd.isna(q1) or pd.isna(q3):
            continue
        if np.isclose(q3 - q1, 0.0):
            nzv_numeric.append(c)
    report["near_zero_variance_numeric"] = nzv_numeric

    # Scale & distribution diagnostics (numeric only)
    desc = X[num_cols].describe().T if num_cols else pd.DataFrame()
    if not desc.empty:
        desc["iqr"] = desc["75%"] - desc["25%"]
        # Robust z-score count beyond |3.5|
        med = X[num_cols].median()
        mad = (X[num_cols] - med).abs().median().replace(0, np.nan)
        robust_z = (X[num_cols] - med).div(mad).abs()
        outlier_counts = (robust_z > 3.5).sum().sort_values(ascending=False)
        report["outlier_counts_numeric"] = outlier_counts[outlier_counts > 0]
        report["numeric_summary"] = desc[["mean", "std", "min", "25%", "50%", "75%", "max", "iqr"]]

    # Correlation structure (numeric only)
    high_corr_pairs = []
    if len(num_cols) >= 2:
        corr = X[num_cols].corr().abs()
        upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
        hits = np.where((upper >= corr_threshold))
        for i, j in zip(*hits):
            high_corr_pairs.append((num_cols[i], num_cols[j], upper.iloc[i, j]))
    report["high_corr_pairs"] = sorted(high_corr_pairs, key=lambda t: t[2], reverse=True)

    # Cardinality of each feature
    cardinality = X.nunique(dropna=True).sort_values()
    report["cardinality"] = cardinality

    # Potential ID-like columns: nearly all values unique (≥95% of rows unique)
    id_like_cols = list(cardinality[cardinality >= 0.95*len(X)].index)
    report["id_like_cols"] = id_like_cols

    # Sequential/monotone with constant step (e.g., 2,3,4,...): likely row index in disguise
    seq_like_cols = []
    for c in X.columns:
        s = X[c]
        if pd.api.types.is_numeric_dtype(s):
            nonna = s.dropna()
            if len(nonna) > 2 and nonna.is_monotonic_increasing:
                diffs = nonna.diff().dropna()
                if diffs.nunique() == 1:
                    seq_like_cols.append(c)
    report["sequential_like_cols"] = seq_like_cols

    # Class distribution + imbalance summary
    counts = y.value_counts().sort_index()
    pct = (100 * counts / len(y)).round(2)
    max_min_ratio = (counts.max() / counts.min()) if counts.min() > 0 else np.inf
    # Entropy (base-2)
    p = counts / counts.sum()
    entropy = -(p * np.log2(p)).sum()
    report["class_distribution"] = pd.DataFrame({"count": counts, "pct": pct})
    report["imbalance_max_min_ratio"] = max_min_ratio
    report["class_entropy_bits"] = float(entropy)

    print("DATA AUDIT SUMMARY")
    print(f"Rows: {len(X):,} | Features: {X.shape[1]} (numeric={len(num_cols)}, binary={len(bin_cols)})")
    print(f"Missing rows (any missing): {report['missing_rows_nonzero']:,}")
    print(f"Columns with missing values: {len(report['missing_cols_nonzero'])}")
    if len(report["missing_cols_nonzero"]) > 0:
        print(report["missing_cols_nonzero"].head(10))

    print(f"\nDuplicate rows: {report['duplicate_rows']:,}")
    print(f"Zero-variance columns: {len(report['zero_variance_cols'])}")
    if report["zero_variance_cols"][:10]:
        print(report["zero_variance_cols"][:10])

    print(f"\nNear-zero-variance numeric: {len(report['near_zero_variance_numeric'])}")
    if report["near_zero_variance_numeric"][:10]:
        print(report["near_zero_variance_numeric"][:10])

    if "outlier_counts_numeric" in report and not report["outlier_counts_numeric"].empty:
        print("\nTop numeric columns with robust outliers (>|3.5| z):")
        print(report["outlier_counts_numeric"].head(10))

    print(f"\nHigh-correlation numeric pairs @|r|≥0.95: {len(report['high_corr_pairs'])}")
    for trip in report["high_corr_pairs"][:5]:
        print(f"  {trip[0]} ~ {trip[1]} |r|={trip[2]:.3f}")

    print("\nClass distribution:\n", report["class_distribution"])
    print(f"Imbalance max/min ratio: {report['imbalance_max_min_ratio']:.2f}")
    print(f"Class entropy (bits): {report['class_entropy_bits']:.3f}")

    low_card = list(cardinality[cardinality <= 1].index)
    print(f"\nLow-cardinality (nunique ≤ 1): {low_card}")
    if id_like_cols:
        print(f"Potential ID-like columns (≈ unique per row): {id_like_cols[:8]}")
    if seq_like_cols:
        print(f"Sequential/monotone step-constant columns: {seq_like_cols[:8]}")


    return report

audit = audit_dataset(X, y, NUM_COLS, BIN_COLS, corr_threshold=0.95)


DATA AUDIT SUMMARY
Rows: 581,012 | Features: 58 (numeric=14, binary=44)
Missing rows (any missing): 298
Columns with missing values: 1
Slope    298
dtype: int64

Duplicate rows: 0
Zero-variance columns: 1
['Water_Level']

Near-zero-variance numeric: 1
['Water_Level']

Top numeric columns with robust outliers (>|3.5| z):
Vertical_Distance_To_Hydrology        57894
Horizontal_Distance_To_Fire_Points    41988
Hillshade_9am                         30893
Horizontal_Distance_To_Hydrology      29620
Elevation                             27133
Hillshade_Noon                        24203
Horizontal_Distance_To_Roadways       22566
Slope                                 18962
Hillshade_3pm                         14852
dtype: int64

High-correlation numeric pairs @|r|≥0.95: 1
  Aspect ~ Facet |r|=1.000

Class distribution:
              count    pct
Cover_Type               
1           211840  36.46
2           283301  48.76
3            35754   6.15
4             2747   0.47
5             9493 

In [ ]:
# Drop features due to data quality issues from assignment
to_drop = []

# Perfect collinearity: keep Aspect, drop Facet
if "Aspect" in X.columns and "Facet" in X.columns:
    to_drop.append("Facet")

# Zero-variance columns
zero_var = [c for c, k in X.nunique(dropna=True).items() if k <= 1]
to_drop += zero_var

# Observation_ID
if "Observation_ID" in X.columns:
    nunq = X["Observation_ID"].nunique(dropna=True)
    if nunq <= 1 or nunq >= 0.95*len(X):
        to_drop.append("Observation_ID")
    else:
        s = X["Observation_ID"].dropna()
        if len(s) > 2 and s.is_monotonic_increasing and s.diff().dropna().nunique() == 1:
            to_drop.append("Observation_ID")

# Pure noise feature
DROP_NOISY_INCLINATION = True
if DROP_NOISY_INCLINATION and "Inclination" in X.columns:
    to_drop.append("Inclination")

# Apply drops
to_drop = [c for c in dict.fromkeys(to_drop) if c in X.columns]
X = X.drop(columns=to_drop).copy()
print("Dropped columns:", to_drop)


Dropped columns: ['Facet', 'Water_Level', 'Observation_ID', 'Inclination']


In [6]:
TRUE_TOKENS  = {"1","true","t","yes","y","pos","positive","on"}
FALSE_TOKENS = {"0","false","f","no","n","neg","negative","off"}

def is_semantic_binary(s):
    u = pd.unique(s.dropna().astype(str).str.strip().str.lower())
    return len(u) == 2 and ((u[0] in TRUE_TOKENS and u[1] in FALSE_TOKENS) or
                            (u[0] in FALSE_TOKENS and u[1] in TRUE_TOKENS))

def mapping_for_binary(s):
    u = pd.unique(s.dropna().astype(str).str.strip())
    low = {v: v.lower() for v in u}
    a, b = u[0], u[1]
    if low[a] in TRUE_TOKENS and low[b] in FALSE_TOKENS: return {a:1, b:0}
    if low[a] in FALSE_TOKENS and low[b] in TRUE_TOKENS: return {a:0, b:1}
    # deterministic fallback
    a1, b1 = sorted([(a,low[a]),(b,low[b])], key=lambda t:t[1])
    return {b1[0]:1, a1[0]:0}

binary_string_cols = [c for c in X.columns if X[c].dtype=='object' and is_semantic_binary(X[c])]
binary_mappings = {}
for c in binary_string_cols:
    m = mapping_for_binary(X[c]); X[c] = X[c].map(m).astype("Int8"); binary_mappings[c]=m

print("Binary string→{0,1} mappings:", {k: v for k,v in list(binary_mappings.items())[:5]})


Binary string→{0,1} mappings: {'Soil_Type1': {'positive': 1, 'negative': 0}}


In [ ]:
for c in X.columns:
    s = X[c]
    if is_numeric_dtype(s):
        vals = pd.unique(s.dropna())
        if len(vals) > 0 and set(map(float, vals)) <= {0.0, 1.0}:
            X[c] = s.astype('int8', copy=False)

def split_num_bin(df: pd.DataFrame):
    num_cols, bin_cols = [], []
    for col in df.columns:
        s = df[col]
        if is_numeric_dtype(s):
            vals = pd.unique(s.dropna())
            if len(vals) > 0 and set(map(float, vals)) <= {0.0, 1.0}:
                bin_cols.append(col)
            else:
                num_cols.append(col)
        else:
            if s.dropna().nunique() == 2:
                bin_cols.append(col)
    return num_cols, bin_cols

NUM_COLS, BIN_COLS = split_num_bin(X)
print(f"[Fixed split] numeric: {len(NUM_COLS)} | binary: {len(BIN_COLS)}")
print("Sample numerics:", NUM_COLS[:8])
print("Sample binaries:", BIN_COLS[:8])

[Fixed split] numeric: 10 | binary: 44
Sample numerics: ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon']
Sample binaries: ['Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4']


In [ ]:

num_cols = [c for c in X.columns if np.issubdtype(X[c].dtype, np.number) and set(pd.unique(X[c].dropna())) - {0,1}]
bin_cols = [c for c in X.columns if np.issubdtype(X[c].dtype, np.number) and set(pd.unique(X[c].dropna())) <= {0,1}]

has_missing = X.isna().any().any()

knn_pre = ColumnTransformer([
    ("num", Pipeline([("impute", SimpleImputer(strategy="median")),
                      ("scale", RobustScaler(quantile_range=(25,75)))]), num_cols),
    ("bin", Pipeline([("impute", SimpleImputer(strategy="most_frequent"))]), bin_cols),
], verbose_feature_names_out=False)

knn_clf = Pipeline([("prep", knn_pre), ("clf", KNeighborsClassifier())])

tree_pre = ("passthrough" if not has_missing else
            ColumnTransformer([
                ("num", SimpleImputer(strategy="median"), num_cols),
                ("bin", SimpleImputer(strategy="most_frequent"), bin_cols),
            ], verbose_feature_names_out=False))

tree_clf = Pipeline([("prep", tree_pre), ("clf", DecisionTreeClassifier(random_state=42))])


In [ ]:
baseline_scoring = {
    "macro_f1": make_scorer(f1_score, average="macro"),
    "balanced_acc": make_scorer(balanced_accuracy_score),
    "kappa": make_scorer(cohen_kappa_score),
    "mcc": make_scorer(matthews_corrcoef),
}
baseline_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=2025)

def summarize_cv(cvres: dict) -> pd.DataFrame:
    return (
        pd.DataFrame({m: [np.mean(cvres[f"test_{m}"]), np.std(cvres[f"test_{m}"])] for m in baseline_scoring})
          .T.rename(columns={0:"mean", 1:"std"}).round(4)
    )

print("=== Baseline k-NN (defaults) ===")
knn_base = cross_validate(knn_clf, X, y, cv=baseline_cv, scoring=baseline_scoring, n_jobs=-1, return_train_score=False)
print(summarize_cv(knn_base))

print("\n=== Baseline Decision Tree (defaults) ===")
tree_base = cross_validate(tree_clf, X, y, cv=baseline_cv, scoring=baseline_scoring, n_jobs=-1, return_train_score=False)
print(summarize_cv(tree_base))


=== Baseline k-NN (defaults) ===
                mean     std
macro_f1      0.8810  0.0024
balanced_acc  0.8698  0.0030
kappa         0.8901  0.0017
mcc           0.8902  0.0016

=== Baseline Decision Tree (defaults) ===
                mean     std
macro_f1      0.9060  0.0034
balanced_acc  0.9049  0.0033
kappa         0.9079  0.0015
mcc           0.9079  0.0015


In [ ]:
# Empirical Procedure: 10x5 nested CV with halving, metrics, logging, Wilcoxon 
scoring = {
    "macro_f1": make_scorer(f1_score, average="macro"),
    "accuracy": make_scorer(accuracy_score),
    "balanced_acc": make_scorer(balanced_accuracy_score),
    "kappa": make_scorer(cohen_kappa_score),
    "mcc": make_scorer(matthews_corrcoef),
    "macro_precision": make_scorer(precision_score, average="macro", zero_division=0),
    "macro_recall": make_scorer(recall_score, average="macro", zero_division=0),
}
primary_key = "macro_f1"

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=2025)
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)

rng = np.random.RandomState(42)
class_labels = np.sort(y.unique())

knn_space = {
    "clf__n_neighbors": list(range(1, 56)),
    "clf__weights": ["uniform", "distance"],
    "clf__metric": ["minkowski", "chebyshev"],
    "clf__p": [1, 2],
}

tree_space = {
    "clf__max_depth": [None] + list(range(3, 41)),
    "clf__min_samples_leaf": [1, 2, 5, 10, 20, 50],
    "clf__min_samples_split": [2, 5, 10, 20, 50],
    "clf__max_features": [None, "sqrt", "log2"] + list(np.round(np.linspace(0.2, 1.0, 9), 2)),
    "clf__ccp_alpha": np.linspace(0.0, 0.02, 21),
    "clf__class_weight": [None, "balanced"],
}

def sample_candidates(space, n_candidates, rng):
    return list(ParameterSampler(space, n_iter=n_candidates, random_state=rng))

def nested_run(label, pipe, space, n_candidates=200, use_halving=True, factor=3, outer_splits=None):
    print(f"\n=== Nested CV [{label}] | outer=10, inner=5 | method={'HalvingRandomSearchCV' if use_halving else 'RandomizedSearchCV'} ===")
    outer_recs = []
    best_params_per_fold = []
    per_fold_confmats = []
    per_fold_perclassF1 = []

    candidates = sample_candidates(space, n_candidates=n_candidates, rng=rng)

    for fold_idx, (tr, te) in enumerate(outer_splits, 1):
        X_tr, X_te = X.iloc[tr], X.iloc[te]
        y_tr, y_te = y.iloc[tr], y.iloc[te]

        if use_halving:
            search = HalvingRandomSearchCV(
                estimator=pipe,
                param_distributions=space,
                resource="n_samples",
                factor=factor,
                scoring="f1_macro",
                cv=inner_cv,
                random_state=42,
                n_jobs=-1,
                verbose=0
            )
        else:
            search = RandomizedSearchCV(
                estimator=pipe,
                param_distributions=space,
                n_iter=n_candidates,
                scoring=scoring,
                refit=primary_key,
                cv=inner_cv,
                random_state=42,
                n_jobs=-1,
                verbose=0
            )

        t0 = time.perf_counter()
        search.fit(X_tr, y_tr)
        fit_time = time.perf_counter() - t0

        best_params_per_fold.append(search.best_params_)

        y_tr_pred = search.best_estimator_.predict(X_tr)
        train_acc = accuracy_score(y_tr, y_tr_pred)

        t1 = time.perf_counter()
        y_pred = search.best_estimator_.predict(X_te)
        pred_time = time.perf_counter() - t1
        pred_time_per_sample = pred_time / len(X_te)

        rec = {
            "fold": fold_idx,
            "macro_f1": f1_score(y_te, y_pred, average="macro"),
            "accuracy": accuracy_score(y_te, y_pred),
            "balanced_acc": balanced_accuracy_score(y_te, y_pred),
            "kappa": cohen_kappa_score(y_te, y_pred),
            "mcc": matthews_corrcoef(y_te, y_pred),
            "macro_precision": precision_score(y_te, y_pred, average="macro", zero_division=0),
            "macro_recall": recall_score(y_te, y_pred, average="macro", zero_division=0),
            "fit_time_s": fit_time,
            "pred_time_per_sample_ms": 1000 * pred_time_per_sample,
            "train_acc": train_acc,
        }
        outer_recs.append(rec)

        cm = confusion_matrix(y_te, y_pred, labels=class_labels)
        per_fold_confmats.append(cm)
        per_class_f1 = f1_score(y_te, y_pred, average=None, labels=class_labels)
        per_fold_perclassF1.append(per_class_f1)

        print(f"Fold {fold_idx:02d} | best params -> {search.best_params_}")
        print("  Test macro-F1: {:.4f} | BalAcc: {:.4f} | κ: {:.4f} | MCC: {:.4f} | Train acc: {:.4f}".format(
            rec["macro_f1"], rec["balanced_acc"], rec["kappa"], rec["mcc"], train_acc
        ))
        print("  Confusion matrix (rows=true, cols=pred):\n", pd.DataFrame(cm, index=class_labels, columns=class_labels))
        print("  Per-class F1:", dict(zip(class_labels, np.round(per_class_f1, 4))))

    df_outer = pd.DataFrame(outer_recs)
    summary = df_outer.describe().loc[["mean", "std"]].round(4)
    print("\n=== Outer-fold summary [{}] (mean ± std) ===".format(label))
    print(summary[["macro_f1","accuracy","balanced_acc","kappa","mcc","macro_precision","macro_recall","fit_time_s","pred_time_per_sample_ms","train_acc"]])

    agg_cm = np.sum(np.stack(per_fold_confmats, axis=0), axis=0)
    agg_cm_df = pd.DataFrame(agg_cm, index=class_labels, columns=class_labels)

    per_class_f1_mean = np.mean(np.vstack(per_fold_perclassF1), axis=0)
    print("\nPer-class F1 (mean over folds):", dict(zip(class_labels, np.round(per_class_f1_mean, 4))))

    return df_outer, best_params_per_fold, agg_cm_df, per_class_f1_mean

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=2025)
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)

outer_splits = list(outer_cv.split(X, y))
knn_results, knn_best, knn_cm, knn_pcF1 = nested_run("k-NN", knn_clf, knn_space, n_candidates=200, use_halving=True, factor=3, outer_splits=outer_splits)
tree_results, tree_best, tree_cm, tree_pcF1 = nested_run("Decision Tree", tree_clf, tree_space, n_candidates=300, use_halving=True, factor=3, outer_splits=outer_splits)

knn_scores = knn_results.sort_values("fold")["macro_f1"].to_numpy()
tree_scores = tree_results.sort_values("fold")["macro_f1"].to_numpy()
diffs = knn_scores - tree_scores

stat, p = wilcoxon(knn_scores, tree_scores, alternative="two-sided", zero_method="wilcox")
z = norm.isf(p/2.0) * np.sign(np.mean(diffs))
r = z / np.sqrt(len(diffs))
median_diff = np.median(diffs)

print("\n=== Wilcoxon signed-rank (macro-F1, k-NN minus Tree) ===")
print(f"Statistic={stat:.3f}, p-value={p:.4g}, effect size r={r:.3f}")
print(f"Median(kNN-Tree) diff: {median_diff:.4f}")
print("Conclusion:", "Significant difference at α=0.05." if p < 0.05 else "No significant difference at α=0.05.")

def fold_summary_table(df, label):
    cols = ["macro_f1","accuracy","balanced_acc","kappa","mcc","macro_precision","macro_recall","fit_time_s","pred_time_per_sample_ms","train_acc"]
    s = df.describe().loc[["mean","std"], cols].round(4)
    s.index = pd.MultiIndex.from_product([[label], s.index])
    return s

report_table = pd.concat([fold_summary_table(knn_results, "k-NN"),
                          fold_summary_table(tree_results, "Decision Tree")])

print("\n=== Table A draft (mean/std over outer folds) ===")
print(report_table)

print("\n=== Table C draft: best params per outer fold ===")
print(pd.DataFrame({"fold": range(1, len(knn_best)+1),
                    "kNN_best": knn_best,
                    "Tree_best": tree_best}))



=== Nested CV [k-NN] | outer=10, inner=5 | method=HalvingRandomSearchCV ===


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 440 is smaller than n_iter=7470. Running 440 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fold 01 | best params -> {'clf__weights': 'distance', 'clf__p': 2, 'clf__n_neighbors': 1, 'clf__metric': 'minkowski'}
  Test macro-F1: 0.8881 | BalAcc: 0.8911 | κ: 0.8996 | MCC: 0.8996 | Train acc: 1.0000
  Confusion matrix (rows=true, cols=pred):
        1      2     3    4    5     6     7
1  19900   1175     1    0   17     4    87
2   1162  26858    79    0  141    79    12
3      2     62  3271   40   13   187     0
4      0      0    33  224    0    18     0
5     28    133    12    0  770     4     2
6      3     65   148   20    9  1492     0
7     90     10     0    0    1     0  1950
  Per-class F1: {np.int64(1): np.float64(0.9394), np.int64(2): np.float64(0.9485), np.int64(3): np.float64(0.9189), np.int64(4): np.float64(0.8014), np.int64(5): np.float64(0.8105), np.int64(6): np.float64(0.8475), np.int64(7): np.float64(0.9508)}


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 440 is smaller than n_iter=7470. Running 440 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fold 02 | best params -> {'clf__weights': 'distance', 'clf__p': 1, 'clf__n_neighbors': 4, 'clf__metric': 'minkowski'}
  Test macro-F1: 0.8974 | BalAcc: 0.8909 | κ: 0.9074 | MCC: 0.9074 | Train acc: 1.0000
  Confusion matrix (rows=true, cols=pred):
        1      2     3    4    5     6     7
1  19930   1138     2    0   25     1    88
2    992  27071    58    0  111    78    20
3      1     69  3307   18    9   172     0
4      0      0    43  217    0    15     0
5     20    153     8    0  762     5     1
6      3     56   143   13    3  1519     0
7     91     15     0    0    0     0  1945
  Per-class F1: {np.int64(1): np.float64(0.9441), np.int64(2): np.float64(0.9527), np.int64(3): np.float64(0.9267), np.int64(4): np.float64(0.8298), np.int64(5): np.float64(0.8198), np.int64(6): np.float64(0.8614), np.int64(7): np.float64(0.9476)}


...

Fold 09 | best params -> {'clf__weights': 'distance', 'clf__p': 1, 'clf__n_neighbors': 4, 'clf__metric': 'minkowski'}
  Test macro-F1: 0.9004 | BalAcc: 0.8923 | κ: 0.9056 | MCC: 0.9057 | Train acc: 1.0000
  Confusion matrix (rows=true, cols=pred):
        1      2     3    4    5     6     7
1  19882   1192     1    0   15     5    89
2   1016  27079    74    0   88    63    10
3      2     61  3309   26    6   171     0
4      0      0    40  222    0    13     0
5     15    140     6    0  781     7     0
6      2     82   169    6    2  1476     0
7     89     19     0    0    2     0  1941
  Per-class F1: {np.int64(1): np.float64(0.9425), np.int64(2): np.float64(0.9518), np.int64(3): np.float64(0.9225), np.int64(4): np.float64(0.8393), np.int64(5): np.float64(0.8475), np.int64(6): np.float64(0.8502), np.int64(7): np.float64(0.9489)}


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 440 is smaller than n_iter=7470. Running 440 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fold 10 | best params -> {'clf__weights': 'distance', 'clf__p': 1, 'clf__n_neighbors': 3, 'clf__metric': 'minkowski'}
  Test macro-F1: 0.8996 | BalAcc: 0.8957 | κ: 0.9030 | MCC: 0.9030 | Train acc: 1.0000
  Confusion matrix (rows=true, cols=pred):
        1      2     3    4    5     6     7
1  19852   1229     4    0   20     5    74
2   1080  26999    71    0  100    70    10
3      0     84  3283   30    9   169     0
4      0      1    33  226    0    15     0
5     23    129    10    0  783     4     0
6      6     74   140   12    3  1502     0
7     94      9     0    0    0     0  1948
  Per-class F1: {np.int64(1): np.float64(0.94), np.int64(2): np.float64(0.9497), np.int64(3): np.float64(0.9227), np.int64(4): np.float64(0.8324), np.int64(5): np.float64(0.8401), np.int64(6): np.float64(0.8578), np.int64(7): np.float64(0.9542)}

=== Outer-fold summary [k-NN] (mean ± std) ===
      macro_f1  accuracy  balanced_acc   kappa     mcc  macro_precision  macro_recall  fit_time_s  \
mean

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Fold 10 | best params -> {'clf__min_samples_split': 5, 'clf__min_samples_leaf': 5, 'clf__max_features': np.float64(0.6), 'clf__max_depth': 35, 'clf__class_weight': None, 'clf__ccp_alpha': np.float64(0.0)}
  Test macro-F1: 0.8797 | BalAcc: 0.8711 | κ: 0.8799 | MCC: 0.8799 | Train acc: 0.9676
  Confusion matrix (rows=true, cols=pred):
        1      2     3    4    5     6     7
1  19609   1442     5    0   18     6   104
2   1479  26575    91    0  111    55    19
3      8     93  3279   40   11   144     0
4      0      1    39  216    0    19     0
5     25    169    16    0  733     6     0
6      7    103   176    9    2  1440     0
7    126     17     0    0    0     0  1908
  Per-class F1: {np.int64(1): np.float64(0.9241), np.int64(2): np.float64(0.9369), np.int64(3): np.float64(0.9132), np.int64(4): np.float64(0.8), np.int64(5): np.float64(0.8037), np.int64(6): np.float64(0.8453), np.int64(7): np.float64(0.9348)}

=== Outer-fold summary [Decision Tree] (mean ± std) ===
      macr

In [ ]:
pd.set_option("display.width", 300)
pd.set_option("display.max_colwidth", None)

In [14]:
print("\n=== Table C draft: best params per outer fold ===")
print(pd.DataFrame({"fold": range(1, len(knn_best)+1),
                    "kNN_best": knn_best,
                    "Tree_best": tree_best}))


=== Table C draft: best params per outer fold ===
   fold                                                                                      kNN_best                                                                                                                                                             Tree_best
0     1  {'clf__weights': 'distance', 'clf__p': 2, 'clf__n_neighbors': 1, 'clf__metric': 'minkowski'}           {'clf__min_samples_split': 2, 'clf__min_samples_leaf': 1, 'clf__max_features': 0.8, 'clf__max_depth': 40, 'clf__class_weight': None, 'clf__ccp_alpha': 0.0}
1     2  {'clf__weights': 'distance', 'clf__p': 1, 'clf__n_neighbors': 4, 'clf__metric': 'minkowski'}  {'clf__min_samples_split': 5, 'clf__min_samples_leaf': 1, 'clf__max_features': None, 'clf__max_depth': None, 'clf__class_weight': 'balanced', 'clf__ccp_alpha': 0.0}
2     3  {'clf__weights': 'distance', 'clf__p': 1, 'clf__n_neighbors': 4, 'clf__metric': 'minkowski'}           {'clf__min_samples_split': 2, 'cl

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path

outdir = Path("./figures")
outdir.mkdir(parents=True, exist_ok=True)

def save_close(fig, name):
    path = outdir / name
    fig.savefig(path, dpi=300, bbox_inches="tight")
    plt.close(fig)
    print("Saved:", path)

knn_f1 = knn_results.sort_values("fold")["macro_f1"].to_numpy()
tree_f1 = tree_results.sort_values("fold")["macro_f1"].to_numpy()

fig = plt.figure(figsize=(5.4, 4.6))
ax = plt.gca()
bp = ax.boxplot([knn_f1, tree_f1], labels=["k-NN", "Decision Tree"], showmeans=True)

x_jitter = 0.06
x_pos = np.array([1, 2])
ax.scatter(np.full_like(knn_f1, x_pos[0]) + (np.random.rand(len(knn_f1))-0.5)*x_jitter, knn_f1, s=18, alpha=0.7)
ax.scatter(np.full_like(tree_f1, x_pos[1]) + (np.random.rand(len(tree_f1))-0.5)*x_jitter, tree_f1, s=18, alpha=0.7)
ax.set_ylabel("Macro-F1")
ax.set_title("")
save_close(fig, "boxplot_macroF1_kNN_vs_Tree.png")

fig = plt.figure(figsize=(6.0, 4.4))
ax = plt.gca()
fold_idx = knn_results.sort_values("fold")["fold"].to_numpy()
ax.plot(fold_idx, knn_f1, marker="o", label="k-NN")
ax.plot(fold_idx, tree_f1, marker="s", label="Decision Tree")
for i in range(len(fold_idx)):
    ax.plot([fold_idx[i], fold_idx[i]], [tree_f1[i], knn_f1[i]], alpha=0.5)  # connector
ax.set_xlabel("Outer fold")
ax.set_ylabel("Macro-F1")
ax.set_title("")
ax.legend()
save_close(fig, "paired_macroF1_per_fold.png")

def row_normalise(df):
    sums = df.sum(axis=1).replace(0, np.nan)
    return df.div(sums, axis=0).fillna(0.0)

def annotate_matrix(ax, mat, fmt="{:.1f}%", cutoff=1.0):
    """
    Write percentages into cells. 'cutoff' is percent threshold (e.g., 1.0 => only >=1% shown).
    """
    nrows, ncols = mat.shape
    for i in range(nrows):
        for j in range(ncols):
            val = mat[i, j]*100.0
            if val >= cutoff:
                ax.text(j, i, fmt.format(val), ha="center", va="center", color="white")

def plot_confusion(cm_df, title, filename, labels, show_diag=True, cutoff=1.0):
    """
    If show_diag=False, zero out the diagonal to emphasise misclassifications.
    cutoff = percentage threshold below which annotations are hidden.
    """
    cm_norm = row_normalise(cm_df).values.copy()
    if not show_diag:
        np.fill_diagonal(cm_norm, 0.0)

    fig = plt.figure(figsize=(6.0, 5.2))
    ax = plt.gca()
    im = ax.imshow(cm_norm, aspect="auto")
    ax.set_title(title)
    ax.set_xlabel("Predicted class")
    ax.set_ylabel("True class")
    ax.set_xticks(range(len(labels)))
    ax.set_yticks(range(len(labels)))
    ax.set_xticklabels(labels, rotation=45, ha="right")
    ax.set_yticklabels(labels)
    cb = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cb.set_label("Row-normalised frequency")
    # Minor-grid to separate cells
    ax.set_xticks(np.arange(-0.5, len(labels), 1), minor=True)
    ax.set_yticks(np.arange(-0.5, len(labels), 1), minor=True)
    ax.grid(which="minor", linestyle="-", linewidth=0.25)

    annotate_matrix(ax, cm_norm, cutoff=cutoff)
    save_close(fig, filename)

plot_confusion(knn_cm,
               "",
               "confusion_kNN_rownorm_annotated.png",
               class_labels, show_diag=True, cutoff=1.0)

plot_confusion(knn_cm,
               "",
               "confusion_kNN_misclass_only.png",
               class_labels, show_diag=False, cutoff=0.5)

plot_confusion(tree_cm,
               "",
               "confusion_Tree_rownorm_annotated.png",
               class_labels, show_diag=True, cutoff=1.0)

plot_confusion(tree_cm,
               "",
               "confusion_Tree_misclass_only.png",
               class_labels, show_diag=False, cutoff=0.5)


/var/folders/np/k3jsjm6n5xn7qyzm8sr6j2tr0000gn/T/ipykernel_78596/3696295677.py:30: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = ax.boxplot([knn_f1, tree_f1], labels=["k-NN", "Decision Tree"], showmeans=True)


Saved: figures2/boxplot_macroF1_kNN_vs_Tree.png
Saved: figures2/paired_macroF1_per_fold.png
Saved: figures2/confusion_kNN_rownorm_annotated.png
Saved: figures2/confusion_kNN_misclass_only.png
Saved: figures2/confusion_Tree_rownorm_annotated.png
Saved: figures2/confusion_Tree_misclass_only.png
